
# Project
 
## Prof . Constantin Pohl   

###  Chiheb Mhamdi  -   Boltzmann Machine : Build a recommendation system  

In [ ]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [ ]:
movies.columns 

#### 0  : Id of the film 
#### 1  : The name of the film 
#### 2  : Type of the film 

In [ ]:
users.columns 

#### 0  : Id of the User
#### 1  : Gender
#### 2  : Age  
#### 3  : 
#### 4  : Timestamp

In [ ]:
ratings.columns 

#### 0  : Id of the User
#### 1  : Id of the film 
#### 2  : Rating
#### 3  : Timestamp

## Preparing the training set and the test set

In [ ]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [ ]:
training_set

In [ ]:
test_set

## Getting the number of users and movies

In [ ]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))
print(nb_users)
print(nb_movies)

In [ ]:
training_set[:,1][training_set[:,0]==1] 

In [ ]:
training_set[:,2] [training_set[:,0]==1]

## Converting the data into an array with users in lines and movies in columns

In [ ]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors

In [ ]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [ ]:
training_set

## Creating the architecture of the Neural Network

In [ ]:
class RBM():
  #initialisation   
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv) # initialise the weights randomly (simulate a gaussian variable M=0/V=1)
    self.a = torch.randn(1, nh) # bias pour for the hidden layers : 1 row / nh columns 
    self.b = torch.randn(1, nv) #bias for the visible neuronnes : 
  #calculate hidden nodes based on visible nodes 
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
#calculate  visible nodes based on hidden nodes
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
#Train our model : 
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t() #update the weights 
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

## How to evaluate the boltzmann machine ? 

#### There are two ways to evaluate our Boltzmann machine: The RMSE and the mean absolute distance.

### RMSE : 
Le RMSE (Root Mean Squared Error, ou Racine de l'Erreur des Moindres Carrés) est calculée comme étant la racine carrée de la moyenne des différences au carré entre les prédictions et les valeurs réelles.

## Training the RBM


In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2)) # RMSE here
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

## Testing the RBM


In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

With the RMSE, our Boltzmann Machine obtains an error around 0.46. This is different from the average absolute distance. An RMSE of 0.46 does not mean that the average distance between the predictions and the actual values ​​is 0.46. It is necessary to compare with the value that one would obtain if one made predictions in a completely random way, which gives an RMSE of 0.72. Thus, an error of 0.46 corresponds to a success rate of 75%.

## The mean absolute distance 


If you prefer to use the average absolute distance, this is also possible. It's more intuitive.

## Training : 

In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) # Average Distance here
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

## Testing : 

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # Average Distance here
        s += 1.
print('test loss: '+str(test_loss/s))

With this metric, we obtain an average distance of 0.24, which is equivalent to approximately 75% correct predictions.

Thus, our model works well.

To test that 0.25 is 75% success, you can try this test.